# Building a Question-Answering System with Hugging Face Transformers

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, pipeline

id = 'deepset/minilm-uncased-squad2'
tokenizer = AutoTokenizer.from_pretrained(id)
model = TFAutoModelForQuestionAnswering.from_pretrained(id, from_pt=True)
pipe = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [ ]:
question = 'What does NLP stand for?'

context = 'Natural Language Processing, or NLP, encompasses a variety of \
           activities, including text classification, keyword and topic \
           extraction, text summarization, and language translation. The \
           accuracy of NLP models has improved in recent years for a variety \
           of reasons, not the least of which are newer and better ways of \
           converting words and sentences into dense vector representations \
           that incorporate context, and a relatively new neural-network \
           architecture called the transformer that can zero in on the most \
           meaningful words and even differentiate between multiple meanings \
           of the same word.'

pipe(question=question, context=context)

In [ ]:
question = 'When was TensorFlow released?'

context = 'Machine learning isn\'t hard when you have a properly engineered \
           dataset to work with. The reason it\'s not hard is libraries such as \
           Scikit-learn and ML.NET, which reduce complex learning algorithms to \
           a few lines of code. Deep learning isn’t difficult, either, thanks to \
           libraries such as the Microsoft Cognitive Toolkit (CNTK), Theano, \
           and PyTorch. But the library that most of the world has settled on \
           for building neural networks is TensorFlow, an open-source framework \
           created by Google that was released under the Apache License 2.0 in \
           2015.'

pipe(question=question, context=context)['answer']

In [ ]:
question = 'Is Keras part of TensorFlow?'

context = 'The learning curve for TensorFlow is rather steep. Another library \
           named Keras provides a simplified Python interface to TensorFlow \
           and has emerged as the Scikit of deep learning. Keras is all about \
           neural networks. It began life as a stand-alone project in 2015 \
           but was integrated into TensorFlow in 2019. Any code that you write \
           using TensorFlow’s built-in Keras module ultimately executes in \
           (and is optimized for) TensorFlow. Even Google recommends using the \
           Keras API.'

pipe(question=question, context=context)['answer']

In [ ]:
question = 'Is it better to use Keras or TensorFlow to build neural networks?'
pipe(question=question, context=context)['answer']

In [ ]:
import pandas as pd

data = pd.read_csv('Data/passages.csv', header=None)
pd.set_option('display.max_colwidth', None)
data.head()

In [ ]:
from transformers import TFAutoModel

bert_id = 'sebastian-hofstaetter/distilbert-dot-margin_mse-T2-msmarco'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_id) 
bert_model = TFAutoModel.from_pretrained(bert_id, from_pt=True)

In [8]:
def vectorize_text(text):
    tokenized_text = bert_tokenizer(text, return_tensors='tf')
    vectorized_text = bert_model(tokenized_text)[0][:, 0, :][0]
    return vectorized_text

contexts = data[0]
vectorized_contexts = contexts.apply(vectorize_text)

In [9]:
import numpy as np

def get_best_contexts(query, contexts, max_matches=3):
    scores = pd.Series(dtype='object')
    tokenized_query = bert_tokenizer(query, return_tensors='tf')
    vectorized_query = bert_model(tokenized_query)[0][:, 0, :][0]
    
    for idx, item in contexts.iteritems():
        score = np.dot(vectorized_query, item)
        scores = pd.concat([scores, pd.Series(score)], ignore_index=True)

    sorted_scores = scores.sort_values(ascending=False)[:max_matches]
    return list(sorted_scores.index)

In [ ]:
question = 'How many versions of YOLO are there?'
indexes = get_best_contexts(question, vectorized_contexts)

for idx in indexes:
    print(f'{contexts[idx]}\n')

In [ ]:
for idx in indexes:
    output = pipe(question=question, context=contexts[idx], handle_impossible_answer=True)
    
    if output['start'] != output['end']:
        print(output)

In [ ]:
def show_answers(question, contexts, vectorized_contexts):
    indexes = get_best_contexts(question, vectorized_contexts)
    
    for idx in indexes:
        output = pipe(question=question, context=contexts[idx], handle_impossible_answer=True)

        if output['start'] != output['end']:
            print(f'{output["answer"]} ({output["score"]:.1%})')

question = 'What type of neural network supports instance segmentation?'
show_answers(question, contexts, vectorized_contexts)

In [ ]:
question = 'What is YOLO\'s primary weakness?'
show_answers(question, contexts, vectorized_contexts)

In [ ]:
question = 'Is TensorFlow difficult to learn?'
show_answers(question, contexts, vectorized_contexts)